In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [ ]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()
    
    
    # =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
    try:
        # get the login button
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@data-qa="navbar login button"]')))
        if login:
            driver.execute_script("arguments[0].scrollIntoView();", login)
            login.click()
        else:
            pass

    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # complete logiin info
# =====================================================================================

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"            
        user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'.//input[@id="email"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")

    # get password tab
    try:
        provided_pass = "Muhssan7865#"
        password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@id="password"]')))
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}")
        
    # click login
    try:
        log = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@data-qa="login form button"]')))
        if log:
            log.click()
            time.sleep(5)
    except Exception as e:
        print("No log in")
    
    # =====================================================================================
    # handle cookies
    # =====================================================================================
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')))
        if cookie:
            cookie.click()
        else:
            pass
    except Exception as e:
        print("No cookies tab found and the error is {e}")
        
    
    # =====================================================================================
    # get total cars
    # =====================================================================================
    try:
        num_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="listing-card"]')))
        if num_cars:
            # cars_list = num_cars.split(" ")
            cars =  int(len(num_cars)) # get only number
            print(f"Total {cars} of cars found")
    except Exception as e:
        print(f"No number of cars found and error is {e}")
        
    
    # result
    results = []
    car_count = 0
    while car_count < cars: #total_cars
        
        try:
            page_cars= WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="listing-card"]')))
            
            for i in range(len(page_cars)): #total_cars
                if car_count >= cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="listing-card"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    # time.sleep(1)
                    page_cars[i].click()
                    time.sleep(2)

                    details = {}

                    # car name --> done
                    try:
                        title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="hero-headline__title"]/h1'))).text.strip()
                        if title:
                            details['Title'] = title
                        else:
                            details['Title'] = "na"
                    except Exception as e:
                        print(f"No car title {e}")
                    
                    # key facts --> done
                    try:
                        facts_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-info-details__key-facts"]')))
                        driver.execute_script("arguments[0].scrollIntoView();", facts_card)
                        if facts_card:
                            facts_lbl = WebDriverWait(facts_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'h3'))).text.strip()
                            if facts_lbl:
                                # print("Label got")
                                facts_ul = WebDriverWait(facts_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'ul')))
                                if facts_ul:
                                    # print("Ul found")
                                    facts_vals =  WebDriverWait(facts_ul, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'li')))
                                    if facts_vals:
                                        # print("Facts found li")
                                        fact_values = [fact_val.text.strip() for fact_val in facts_vals]
                                        details[facts_lbl] = fact_values
                                    else:
                                        print("No facts data")
                    except Exception as e:
                        print(f"No facts data {e}")
                    
                    # other information --> done
                    try:
                        det_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-info-details__stats"]'))) 
                        if det_card:
                            # print("det found")
                            det_ul = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'ul')))
                            if det_ul:
                                # print("ul found")
                                det_vals =  WebDriverWait(det_ul, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'li')))
                                if det_vals:
                                    lbl_lst = []
                                    for li in det_vals:
                                        lbls = li.get_attribute("data-qa").split(" ")[3]
                                        lbl_lst.append(lbls)
                                        
                                        spans = li.find_elements(By.XPATH, './/span[@class="icon-text is-capitalized"]')
                                        if spans:
                                            for span in spans:
                                                vals = span.find_element(By.XPATH, './/span[@class="text"]').text.strip()
                                                details[lbls] = vals
                                    # results.append(dict(zip(lbl_lst, vals)))
                    except Exception as e:
                        print(f"No car dets {e}")
                        
                    # location --> done
                    try:
                        loc_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-info-details__location"]'))) 
                        if loc_card:
                            loc_lbl = loc_card.find_element(By.TAG_NAME, 'h6').text.strip()
                            loc_val = loc_card.find_element(By.XPATH, './/span[@class="text"]').text.strip()
                            if loc_lbl and loc_val:
                                details[loc_lbl] = loc_val
                            else:
                                print("No loc infor")
                    except Exception as e:
                        print("No loc infor")
                        
                    # bg dets --> done
                    try:
                        bg_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content background"])[1]'))) 
                        if bg_card:
                            bg_dets = WebDriverWait(bg_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
                            if bg_dets:
                                details['Background'] = bg_dets
                            else:
                                details['Background'] = 'na'
                    except Exception as e:
                        print("No bg card found")
                        
                    # overview dets --> done
                    try:
                        over_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content overview"])[1]'))) 
                        driver.execute_script("arguments[0].scrollIntoView();", over_card)
                        if over_card:
                            over_dets = WebDriverWait(over_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
                            if over_dets:
                                details['Overview'] = over_dets
                            else:
                                details['Overview'] = 'na'
                    except Exception as e:
                        print("No overview card found")
                        
                    # est val and seller --> done
                    try:
                        est_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-about__details"]'))) 
                        if est_card:
                            
                            # set val --> done
                            try:
                                est = WebDriverWait(est_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-about__estimated"]')))
                                if est:
                                    est_lbl = est.find_element(By.TAG_NAME, 'p').text.strip()
                                    est_val = est.find_element(By.XPATH, './/span[@class="text"]').text.strip()
                                    if est_lbl and est_val:
                                        details[est_lbl] = est_val
                            except Exception as e:
                                print("No estimated values")
                                
                            # seller --> done
                            try:
                                sell = WebDriverWait(est_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-about__seller"]')))
                                if sell:
                                    sell_lbl = sell.find_element(By.TAG_NAME, 'p').text.strip()
                                    sell_val = sell.find_element(By.XPATH, './/span[@class="text"]').text.strip()
                                    if sell_lbl and sell_val:
                                        details[sell_lbl] = sell_val
                            except Exception as e:
                                print("No seller values")
                    except Exception as e:
                        print("No seller dets")
                        
                    
                    # Exterior dets and images --> done
                    try:
                        ext_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content exterior"])[1]'))) 
                        driver.execute_script("arguments[0].scrollIntoView();", ext_card)
                        if ext_card:
                            ext_dets = WebDriverWait(ext_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
                            if ext_dets:
                                details['exterior'] = ext_dets
                            else:
                                details['exterior'] = 'na'
                            
                            try:
                                # imgs
                                to_clk = WebDriverWait(ext_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="vehicle-gallery__overlay"]'))) 
                                if to_clk:
                                    driver.execute_script("arguments[0].scrollIntoView();", to_clk)
                                    to_clk.click()
                                    time.sleep(2)
                                    
                                    img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@id="productGallery"])')))
                                    if img_card:
                                        imgs = WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class=" twic-done"]')))
                                        if imgs:
                                            imgs_lst = [img.get_attribute("src") for img in imgs]
                                            details['Exterior_Images'] = ", ".join(imgs_lst)
                                        else:
                                            print("No images ")
                            except Exception as e:
                                print("No exterior images found")   
                            driver.back()     
                    except Exception as e:
                        print("No exterior card found")
                    
                    
                    # Interior dets and images --> done
                    try:
                        intr_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content interior"])[1]'))) 
                        driver.execute_script("arguments[0].scrollIntoView();", intr_card)
                        if intr_card:
                            intr_dets = WebDriverWait(intr_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
                            if intr_dets:
                                details['interior'] = intr_dets
                            else:
                                details['interior'] = 'na'
                            
                            try:
                                # imgs
                                to_clk = WebDriverWait(intr_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="vehicle-gallery__overlay"]'))) 
                                if to_clk:
                                    driver.execute_script("arguments[0].scrollIntoView();", to_clk)
                                    to_clk.click()
                                    # time.sleep(2)
                                    
                                    img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@id="productGallery"])')))
                                    if img_card:
                                        imgs = WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class=" twic-done"]')))
                                        if imgs:
                                            imgs_lst = [img.get_attribute("src") for img in imgs]
                                            details['interior_Images'] = ", ".join(imgs_lst)
                                        else:
                                            print("No images ")
                            except Exception as e:
                                print("No interior images found")      
                            driver.back()   
                    except Exception as e:
                        print("No interior card found")
                      
                    
                    # Mechanical dets and images --> done
                    try:
                        mech_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content mechanical"])[1]'))) 
                        driver.execute_script("arguments[0].scrollIntoView();", mech_card)
                        if mech_card:
                            mech_dets = WebDriverWait(mech_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
                            if mech_dets:
                                details['mechanical'] = mech_dets
                            else:
                                details['mechanical'] = 'na'
                            
                            try:
                                # imgs
                                to_clk = WebDriverWait(mech_card, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@class="vehicle-gallery__overlay"]'))) 
                                if to_clk:
                                    driver.execute_script("arguments[0].scrollIntoView();", to_clk)
                                    to_clk.click()
                                    # time.sleep(2)
                                    
                                    img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@id="productGallery"])')))
                                    if img_card:
                                        imgs = WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class=" twic-done"]')))
                                        if imgs:
                                            imgs_lst = [img.get_attribute("src") for img in imgs]
                                            details['mechanical_Images'] = ", ".join(imgs_lst)
                                        else:
                                            print("No images ")
                            except Exception as e:
                                print("No mechanical images found")    
                            driver.back()    
                    except Exception as e:
                        print("No imechanical card found")
                    
                    
                    # History dets and images --> done
                    try:
                        hist_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content history"])[1]'))) 
                        driver.execute_script("arguments[0].scrollIntoView();", hist_card)
                        if hist_card:
                            hist_dets = WebDriverWait(hist_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
                            if hist_dets:
                                details['History'] = hist_dets
                            else:
                                details['History'] = 'na'
                            
                            try:
                                # imgs
                                to_clk = WebDriverWait(hist_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//a[@class="mt-2"])[1]'))) 
                                if to_clk:
                                    # driver.execute_script("arguments[0].scrollIntoView();", to_clk)
                                    to_clk.click()
                                    # time.sleep(2)
                                    
                                    img_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@id="productGallery"])')))
                                    if img_card:
                                        imgs = WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class=" twic-done"]')))
                                        if imgs:
                                            imgs_lst = [img.get_attribute("src") for img in imgs]
                                            details['History_Images'] = ", ".join(imgs_lst)
                                        else:
                                            print("No images ")
                            except Exception as e:
                                print("No History images found")   
                            driver.back()     
                    except Exception as e:
                        print("No History card found")
                    
                    
                    # Summary dets --> done
                    try:
                        summary_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//article[@data-qa="auction content summary"])[1]'))) 
                        driver.execute_script("arguments[0].scrollIntoView();", summary_card)
                        if summary_card:
                            summary_dets = WebDriverWait(summary_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content__inner"]'))).text.strip()
                            if summary_dets:
                                details['Summary'] = summary_dets
                            else:
                                details['Summary'] = 'na'
                    except Exception as e:
                        print("No summary found")
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
                    
                except Exception as e:
                    print("No more cars to click")
                    break
        except Exception as e:
            print("No more cars")
            break
        
    # time.sleep(5)
    df = pd.DataFrame.from_dict(results)
    df.to_csv("Bonhams.csv", index=False)
    driver.quit()

path = "https://carsonline.bonhams.com/en/auctions/live"
scrape(path)

In [2]:
df = pd.read_csv("Bonhams.csv")
df.columns


Index(['Title', 'Key Facts', 'chassis', 'mileage', 'engine', 'gearbox',
       'color', 'interior', 'steering', 'fuel_type', 'Vehicle location',
       'Background', 'Overview', 'Estimated value', 'Seller', 'exterior',
       'Exterior_Images', 'interior_Images', 'mechanical', 'mechanical_Images',
       'History', 'History_Images', 'Summary'],
      dtype='object')

### Image downloader

### Exterior images

In [3]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Title", "Exterior_Images"]]

def download_images(data, main_folder="Exterior_Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Title"] 
        image_urls = row["Exterior_Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_1_419da4c9-d2d4-4a1e-9503-7d1db9d8192d.jpg
Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_2_ffbfe57e-71fd-432e-bace-055c963e0caa.jpg
Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_3_c56a53de-e62c-43ca-88b7-aa9e2f5ffff4.jpg
Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_4_41500af6-482b-4be5-a84a-88fde7c9fec5.jpg
Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_5_41b3bcfb-1d78-4356-b3f7-fd743e669cf5.jpg
Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_6_d4818531-9ec4-4346-94f7-713a16b42b33.jpg
Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_7_754dcd7f-48fb-4fa5-91fc-f17998d47916.jpg
Downloaded: Exterior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_8_53311794-42cd-4255-91f8-7683aecd7460.jpg


### Interior images

In [4]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Title", "interior_Images"]]

def download_images(data, main_folder="interior_Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Title"] 
        image_urls = row["interior_Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_1_5b60288a-0cbf-49e5-8c95-dc22c874b6f2.jpg
Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_2_516811cb-e4d1-4faa-91be-1d203df1e6b6.jpg
Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_3_32eecfab-d63e-44ae-b8bf-e00b115b5577.jpg
Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_4_9e5b1095-ef92-4361-8a38-8c52b6c40c8a.jpg
Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_5_921de270-5789-450f-aed1-f038f5e852fd.jpg
Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_6_02be8225-f7ef-4e00-af0c-cbc6b8bd92fe.jpg
Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_7_e322b44f-ea32-4001-a44b-9a8492f9365b.jpg
Downloaded: interior_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_8_1733dc23-640c-4b50-a18d-9353480fb3ea.jpg


### Mechanical images

In [5]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Title", "mechanical_Images"]]

def download_images(data, main_folder="mechanical_Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Title"] 
        image_urls = row["mechanical_Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_1_5ac6aedd-f2dc-4ff0-aee3-ffb082e014af.jpg
Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_2_84374909-b9b3-40e8-8f16-b9d0512429cf.jpg
Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_3_7909c46e-93c6-4b59-b98d-082960472958.jpg
Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_4_652813ef-07e3-4cdc-b37c-5bd96f323bfc.jpg
Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_5_8f456415-5f31-4a8d-972e-373a53f427b4.jpg
Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_6_4ba1a053-1a7a-4b50-bfd2-c737f4b58f43.jpg
Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_7_aae04ce1-b14b-4162-8895-36d6cb502841.jpg
Downloaded: mechanical_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_8_667e93de-5167-4377-b764-a

### History images

In [6]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Title", "History_Images"]]

def download_images(data, main_folder="History_Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Title"] 
        image_urls = row["History_Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_1_b36535c2-db33-460a-95e0-65b7525aa6c8.jpg
Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_2_5758c20a-39f9-4477-8b9b-6c26cab8dc69.jpg
Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_3_b11456d6-43df-49be-9f46-6899426c60eb.jpg
Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_4_ac73268d-df6c-4bb2-b11c-99a77824feac.jpg
Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_5_498acb89-502a-4270-8f71-3ee72a7fda09.jpg
Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_6_14688958-c4a6-44c1-8e88-2e1d215cd420.jpg
Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_7_da8a0718-768d-42cd-8353-7e22eab40ab8.jpg
Downloaded: History_Images\1966 Chevrolet C10 Rat Rod\1966 Chevrolet C10 Rat Rod_8_1d196e50-e761-440d-8013-4d6d00548473.jpg
Download